### The way to calculate the coastal length of mangroves is:
1.- Buffer extent data by 200m  
2.- Clip the buffered extent with the target feature  
3.- Clip the coastal extent with the clipped buffered extent  
4.- Calculate the length of the clipped coastal extent  
5.- repeat the process for each year extent and each location  

In [1]:
import os
import logging
from pathlib import Path
import requests

import json
import multiprocessing as mp

from shapely.geometry import Polygon, box, mapping
import fiona
import geopandas as gpd
import shapely.speedups
shapely.speedups.enable()
%run utils.ipynb

In [3]:
# LOCAL PATHS
#  FIXME: This will depends from where the notebook kernel is running so be careful
WORK_DIR =Path(os.getcwd())
BASE_DIR = f'{WORK_DIR.parents[3]}/datasets'

# @TODO: Add expected data files source as an environment variable.
assert BASE_DIR == '/home/jovyan/work/datasets', f'{BASE_DIR} is not the correct directory'

# variables
coast_df = gpd.read_file(f'{BASE_DIR}/processed/coastline.gpkg', bbox=[-180, -50, 180, 50])
locations_df = gpd.read_file(f'{BASE_DIR}/processed/locations/locations_v3_not_merged_with_old.gpkg')
mangrove_extent_path = Path(f'{BASE_DIR}/raw/extent-layer-creation/gmw_v3_fnl_mjr_v314.gpkg')
layers = fiona.listlayers(mangrove_extent_path)

coast_df.sindex
locations_df.sindex

rtree.index.Index(bounds=[-179.99998854118687, -58.44946994568893, 179.99998854118684, 74.70884000396096], size=70)

In [10]:
# gpd.overlay(coast_df, locations_df, how='intersection')

In [8]:
gpd.sjoin(locations_df, coast_df, how='left', op='intersects')

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,name,iso,type,area_m2,wdpaid,globalid,perimeter_m,location_idn,coast_length_m,geometry,index_right
0,Port Albert,NZL,wdpa,6.284804e-07,305383.0,None,0.005729,df6769ce-2dab-565f-9988-3cbc4d1fe27b,31.84,"MULTIPOLYGON (((174.43184 -36.27456, 174.43183...",152304.0
1,East Beach,NZL,wdpa,1.339717e-03,304976.0,None,0.837780,fadaf94e-88c6-5f6b-9186-263ac02bc1ab,8165.40,"MULTIPOLYGON (((173.24172 -34.90725, 173.24165...",150182.0
1,East Beach,NZL,wdpa,1.339717e-03,304976.0,None,0.837780,fadaf94e-88c6-5f6b-9186-263ac02bc1ab,8165.40,"MULTIPOLYGON (((173.24172 -34.90725, 173.24165...",151301.0
2,Ngunguru River,NZL,wdpa,1.502996e-05,304437.0,None,0.032190,db8e486b-f23f-5f32-bb4b-81d5f28ca6e7,718.36,"MULTIPOLYGON (((174.48670 -35.64307, 174.48664...",151875.0
3,Kaitoke,NZL,wdpa,1.245820e-04,304209.0,None,0.098260,a8a340fc-a9e1-5af0-9d43-4c8080131ee2,2726.67,"MULTIPOLYGON (((175.71073 -36.87066, 175.71071...",153803.0
...,...,...,...,...,...,...,...,...,...,...,...
3172,Saloum Delta,SEN,aoi,3.361619e-01,NaN,None,2.352446,9e2d8fc4-9ed4-5aea-8220-a200f1c388a7,440446.25,"MULTIPOLYGON (((-16.84289 13.48422, -16.84289 ...",81604.0
3172,Saloum Delta,SEN,aoi,3.361619e-01,NaN,None,2.352446,9e2d8fc4-9ed4-5aea-8220-a200f1c388a7,440446.25,"MULTIPOLYGON (((-16.84289 13.48422, -16.84289 ...",81605.0
3172,Saloum Delta,SEN,aoi,3.361619e-01,NaN,None,2.352446,9e2d8fc4-9ed4-5aea-8220-a200f1c388a7,440446.25,"MULTIPOLYGON (((-16.84289 13.48422, -16.84289 ...",81570.0
3172,Saloum Delta,SEN,aoi,3.361619e-01,NaN,None,2.352446,9e2d8fc4-9ed4-5aea-8220-a200f1c388a7,440446.25,"MULTIPOLYGON (((-16.84289 13.48422, -16.84289 ...",81572.0


In [6]:
mangrove_extent_df = gpd.read_file(mangrove_extent_path, layer=layers[0])#.geometry.to_crs('epsg:3410').buffer(200)

In [7]:
mangrove_extent_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1121419 entries, 0 to 1121418
Data columns (total 2 columns):
 #   Column    Non-Null Count    Dtype   
---  ------    --------------    -----   
 0   PXLVAL    1121419 non-null  int64   
 1   geometry  1121419 non-null  geometry
dtypes: geometry(1), int64(1)
memory usage: 17.1 MB


In [ ]:
mangrove_extent_df.to_crs('epsg:3410').buffer(200)

In [ ]:
gpd.sjoin(coast_df, locations_df, how='left', op='intersects')

In [13]:
test = gpd.sjoin_nearest(coast_df.to_crs('epsg:3410'), mangrove_extent_df.to_crs('epsg:3410'), 
            how='inner', max_distance=200)

NotImplementedError: Currently, only PyGEOS >= 0.10.0 supports `nearest_all`. To use PyGEOS within GeoPandas, you need to install PyGEOS: 'conda install pygeos' or 'pip install pygeos'

In [12]:
test.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 176028 entries, 73 to 277825
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   geometry     176028 non-null  geometry
 1   index_right  176028 non-null  int64   
 2   PXLVAL       176028 non-null  int64   
dtypes: geometry(1), int64(2)
memory usage: 5.4 MB


In [ ]:
for layer_name in layers:
    print(layer_name)
    mangrove_extent_df = gpd.read_file(mangrove_extent_path, layer=layer_name)
    mangrove_extent_df.sindex
    mangrove_extent_df.clip()

In [ ]:
def calculateMangrovesCoastalLength(geometry):
    if geometry:
        return round(coast_df.clip(geometry).to_crs('EPSG:3410').geometry.length.sum(), 2)
    else:
        return None   

In [ ]:
with mp.Pool(mp.cpu_count() - 1) as pool:
    locations_df['coast_length_m'] = pool.map(calculateMangrovesCoastalLength, locations_df['geometry'])